<a href="https://colab.research.google.com/github/rafaelmoczalla/slicing/blob/main/RunningIntervalsInScotty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Intervals in Scotty

This notebook presents experiments on extending and applying running intervals as presented by [Arvind and Jennifer](http://ilpubs.stanford.edu:8090/637/1/2004-15.pdf) to the general stream slicing framework Scotty.

## Dependencies Setup
### Install gradle 7.2

In [6]:
%cd /content
!if [ ! -f "/content/gradle-7.2-bin.zip" ]; then if [ -z $(ls -A /opt/gradle) ]; then cd /content && wget https://services.gradle.org/distributions/gradle-7.2-bin.zip; fi; fi
!if [ ! -d "/opt/gradle" ]; then mkdir /opt/gradle; fi
!if [ -z $(ls -A /opt/gradle) ]; then unzip -d /opt/gradle /content/gradle-7.2-bin.zip; fi
!if [ ! -f "/usr/local/bin/gradle" ]; then ln -s /opt/gradle/gradle-7.2/bin/gradle /usr/local/bin/gradle; fi

/content


## Install C++, Git and Additional Python Header Files for C++

In [10]:
!apt install build-essential git python3-dev
!if [ ! -d "/content/slicing" ]; then git clone https://github.com/rafaelmoczalla/slicing; else cd /content/slicing && git pull; fi
%set_env CPATH=/usr/include/python3.7m
%set_env LD_LIBRARY_PATH=/usr/lib:/usr/lib/python3.7/config-3.7m-x86_64-linux-gnu/
%cd /content/slicing
!gradle build && gradle runTest

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
git is already the newest version (1:2.17.1-1ubuntu0.9).
python3-dev is already the newest version (3.6.7-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/rafaelmoczalla/slicing
   23bf996..d6a1b99  main       -> origin/main
Updating 23bf996..d6a1b99
Fast-forward
 RunningIntervalsInScotty.ipynb | 3454 ++++++++++++++++++++++++++++++++++++++++
 1 file changed, 3454 insertions(+)
 create mode 100644 RunningIntervalsInScotty.ipynb
env: CPATH=/usr/include/python3.7m
env: LD_LIBRARY_PATH=/usr/lib:/usr/lib/python3.7/config-3.7m-x86_64-linux-gnu/
/content/slicing


<==

## Experiments
In the following experiment we compare a simple version of Scotty to running intervals.

In [8]:
%%writefile /content/slicing/Experiments/main.cpp

#define PY_SSIZE_T_CLEAN
#include <iostream>
#include <iomanip>
#include <random>
#include <ctime>
#include <sys/time.h>
#include <Python.h>

#include <bucketSlidingWindowOperator.hpp>
#include <slicedSlidingWindowOperator.hpp>
#include <rollingSlidingWindowOperator.hpp>
#include <csv.hpp>

struct Setup {
    std::string name;
    std::vector<long> wmNr;
    std::vector<long> ePerWm;
    std::vector<long> wLen;
    std::vector<long> sLen;
    std::string to_string() {
        std::stringstream out;

        out << "Setup{" << std::endl << "name=" << name << std::endl;

        out << "wmNr=[";
        for (int i = 0; i < wmNr.size() - 1; i++) {
            out << std::to_string(wmNr[i]) << ", ";
        }
        if (!wmNr.empty())
            out << std::to_string(wmNr.back());
        out << "]," << std::endl;

        out << "ePerWm=[";
        for (int i = 0; i < ePerWm.size() - 1; i++) {
            out << std::to_string(ePerWm[i]) << ", ";
        }
        if (!ePerWm.empty())
            out << std::to_string(ePerWm.back());
        out << "]," << std::endl;

        out << "wLen=[";
        for (int i = 0; i < wLen.size() - 1; i++) {
            out << std::to_string(wLen[i]) << ", ";
        }
        if (!wLen.empty())
            out << std::to_string(wLen.back());
        out << "]," << std::endl;

        out << "sLen=[";
        for (int i = 0; i < sLen.size() - 1; i++) {
            out << std::to_string(sLen[i]) << ", ";
        }
        if (!sLen.empty())
            out << std::to_string(sLen.back());
        out << "]}" << std::endl;

        return out.str();;
    };
};

double get_wall_time() {
    struct timeval t;
    if (gettimeofday(&t, NULL)){
        //  Handle error
        return 0;
    }
    return (double)t.tv_sec + (double)t.tv_usec * .000001;
}

std::string get_now() {
    time_t rawtime;
    struct tm *gmt;
    time(&rawtime);
    gmt = gmtime(&rawtime);

    std::string today("GMT-");
    today += std::to_string(gmt->tm_year + 1900);
    if (gmt->tm_mon + 1 < 10)
        today += "0";
    today += std::to_string(gmt->tm_mon + 1);
    if (gmt->tm_mday < 10)
        today += "0";
    today += std::to_string(gmt->tm_mday);
    if (gmt->tm_hour < 10)
        today += "-0";
    else
        today += "-";
    today += std::to_string(gmt->tm_hour);
    if (gmt->tm_min < 10)
        today += "0";
    today += std::to_string(gmt->tm_min);
    if (gmt->tm_sec < 10)
        today += "0";
    today += std::to_string(gmt->tm_sec);

    return today;
}

void experiment(Csv &csv, long wmNr, long ePerWm, long wLen, long sLen,
        std::vector<double> &bucket, std::vector<double> &sliced, std::vector<double> &rolled) {
    double start;

    std::default_random_engine generator(117);
    std::default_random_engine generatorBucket(117);
    std::default_random_engine generatorSlicing(117);
    std::default_random_engine generatorRolling(117);

    std::uniform_real_distribution<double> randomValue(0.0, 200.0);
    std::uniform_int_distribution<long> randomOutOfOrder(-10, 20);
    std::uniform_int_distribution<long> randomNrOfEventsTilWatermark(0, ePerWm);

    std::cout << "Experimental Environment Initialization" << std::endl;

    long iWm = randomNrOfEventsTilWatermark(generator);
    long numberOfEvents = 0;
    long ts;
    long cWm = 0;
    long lWm = 0;
    double bucketTime;
    double slicingTime;
    double rollingTime;

    for (long wm = 0; wm < wmNr; wm++) {
        for (long e = 0; e < iWm; e++)
            numberOfEvents++;

        iWm = randomNrOfEventsTilWatermark(generator);
    }

    csv << std::to_string(numberOfEvents)
    << std::to_string(wmNr)
    << std::to_string(ePerWm)
    << std::to_string(wLen)
    << std::to_string(sLen);

    std::cout << "Initializing Bucket Sliding Window Experiment" << std::endl;
    // bucketOperator experiment
    iWm = randomNrOfEventsTilWatermark(generatorBucket);
    ts = 10;
    cWm = 0;
    lWm = 0;

    BucketSlidingWindowOperator *bucketOperator = new BucketSlidingWindowOperator(0, wLen, sLen);

    std::cout << "Started Bucket Sliding Window Experiment" << std::endl;
    start = get_wall_time();
    for (long wm = 0; wm < wmNr; wm++) {
        for (long e = 0; e < iWm; e++) {
            bucketOperator->processElement(
                new Event(
                    randomValue(generatorBucket),
                    ts + randomOutOfOrder(generatorBucket)
            ));
            ts += 10;
        }

        cWm = ts + randomOutOfOrder(generatorBucket);
        if (cWm <= lWm)
            cWm = lWm + 1;
        std::vector<Element *> out
                = bucketOperator->processWatermark(new Watermark(cWm));
        lWm = cWm;

        // discard output stream for now
        for (Element *o : out)
            delete o;

        out.clear();
        out.resize(0);

        iWm = randomNrOfEventsTilWatermark(generatorBucket);
    }

    bucketTime = get_wall_time() - start;
    csv << std::to_string(bucketTime);
    bucket.push_back(bucketTime);

    std::cout << "Cleaning memory" << std::endl;
    delete bucketOperator;


    std::cout << "Initializing Sliced Sliding Window Experiment" << std::endl;
    // slicedOperator experiment
    iWm = randomNrOfEventsTilWatermark(generatorSlicing);
    ts = 10;
    cWm = 0;
    lWm = 0;

    SlicedSlidingWindowOperator *slicedOperator = new SlicedSlidingWindowOperator(0, wLen, sLen);

    std::cout << "Started Sliced Sliding Window Experiment" << std::endl;
    start = get_wall_time();
    for (long wm = 0; wm < wmNr; wm++) {
        for (long e = 0; e < iWm; e++) {
            slicedOperator->processElement(
                new Event(
                    randomValue(generatorSlicing),
                    ts + randomOutOfOrder(generatorSlicing)
            ));
            ts += 10;
        }

        cWm = ts + randomOutOfOrder(generatorSlicing);
        if (cWm <= lWm)
            cWm = lWm + 1;
        std::vector<Element *> out
                = slicedOperator->processWatermark(new Watermark(cWm));
        lWm = cWm;

        // discard output stream for now
        for (Element *o : out)
            delete o;

        out.clear();
        out.resize(0);

        iWm = randomNrOfEventsTilWatermark(generatorSlicing);
    }

    slicingTime = get_wall_time() - start;
    csv << std::to_string(slicingTime);
    sliced.push_back(slicingTime);

    std::cout << "Cleaning memory" << std::endl;
    delete slicedOperator;


    std::cout << "Initializing Rolling Sliding Window Experiment" << std::endl;
    // prototype experiment
    iWm = randomNrOfEventsTilWatermark(generatorRolling);
    ts = 10;
    cWm = 0;
    lWm = 0;

    RollingSlidingWindowOperator *prototype = new RollingSlidingWindowOperator(0, wLen, sLen);

    std::cout << "Started Rolling Sliding Window Experiment" << std::endl;
    start = get_wall_time();
    for (long wm = 0; wm < wmNr; wm++) {
        for (long e = 0; e < iWm; e++) {
            prototype->processElement(
                new Event(
                    randomValue(generatorRolling),
                    ts + randomOutOfOrder(generatorRolling)
            ));
            ts += 10;
        }

        cWm = ts + randomOutOfOrder(generatorRolling);
        if (cWm <= lWm)
            cWm = lWm + 1;
        std::vector<Element *> out
                = prototype->processWatermark(new Watermark(cWm));
        lWm = cWm;

        // discard output stream for now
        for (Element *o : out)
            delete o;

        out.clear();
        out.resize(0);
        iWm = randomNrOfEventsTilWatermark(generatorRolling);
    }

    rollingTime = get_wall_time() - start;
    csv << std::to_string(rollingTime);
    rolled.push_back(rollingTime);

    std::cout << "Cleaning memory" << std::endl;
    delete prototype;

    double sP = rollingTime / slicingTime;
    double rP = slicingTime / rollingTime;

    std::cout.precision(3);
    std::cout << std::fixed << "Summary" << std::endl;
    std::cout << "  Number of events processed: " << numberOfEvents << std::endl;
    std::cout << "  Bucket Sliding Window time in sec:  " << bucketTime << std::endl;
    std::cout << "  Sliced Sliding Window time in sec:  " << slicingTime << std::endl;
    std::cout << "  Rolling Sliding Window time in sec: " << rollingTime << std::endl;
    if (sP > rP)
        std::cout << "  Sliced Sliding Window is " << sP << " faster than Rolling Sliding Window" << std::endl;
    else
        std::cout << "  Rolling Sliding Window is " << rP << " faster than Sliced Sliding Window" << std::endl;

    csv << std::endl;
}

int main(int argc, char *argv[]) {
    {
        std::string today = get_now();
        std::string folder = "out/rollingExperiments_" + today;

        std::filesystem::create_directories(folder);

        std::vector<Setup> experiments;

        Setup setup1;
        setup1.name = "Varying Count of Watermarks";
        setup1.wmNr.push_back(10000);
        setup1.wmNr.push_back(20000);
        setup1.wmNr.push_back(30000);
        setup1.wmNr.push_back(40000);
        setup1.wmNr.push_back(50000);
        setup1.wmNr.push_back(60000);
        setup1.ePerWm.push_back(50);
        setup1.wLen.push_back(7200);
        setup1.sLen.push_back(30);
        experiments.push_back(setup1);

        Setup setup2;
        setup2.name = "Varying Count of Events per Watermark";
        setup2.wmNr.push_back(10000);
        setup2.ePerWm.push_back(300);
        setup2.ePerWm.push_back(600);
        setup2.ePerWm.push_back(900);
        setup2.ePerWm.push_back(1200);
        setup2.ePerWm.push_back(1500);
        setup2.ePerWm.push_back(1800);
        setup2.wLen.push_back(7200);
        setup2.sLen.push_back(30);
        experiments.push_back(setup2);

        Setup setup3;
        setup3.name = "Varying Count of Parallel Windows";
        setup3.wmNr.push_back(10000);
        setup3.ePerWm.push_back(50);
        setup3.wLen.push_back(6000);
        setup3.wLen.push_back(12000);
        setup3.wLen.push_back(18000);
        setup3.wLen.push_back(24000);
        setup3.wLen.push_back(30000);
        setup3.wLen.push_back(36000);
        setup3.sLen.push_back(30);
        experiments.push_back(setup3);

        // plot stuff
        wchar_t *plot = Py_DecodeLocale(argv[0], NULL);
        Py_SetProgramName(plot);
        Py_Initialize();
        PyRun_SimpleString("import matplotlib.pyplot as plt");

        // run experiments and plot
        for (Setup setup : experiments) {
            std::cout << std::endl << setup.name << std::endl;
            for (char item : setup.name)
                std::cout << "*";
            std::cout << std::endl;

            Csv csv(folder, setup.name);

            csv << "Event Count"
            << "Watermark Count"
            << "Event Count per Watermark"
            << "Window Length"
            << "Step Length"
            << "Bucket Window Runtime in Sec"
            << "Sliced Window Runtime in Sec"
            << "Rolling Window Runtime in Sec"
            << std::endl;

            std::vector<double> yBucket;
            std::vector<double> ySliced;
            std::vector<double> yRolled;

            for (long wmNr : setup.wmNr)
                for (long ePerWm : setup.ePerWm)
                    for (long wLen : setup.wLen)
                        for (long sLen : setup.sLen) {
                            experiment(csv, wmNr, ePerWm, wLen, sLen, yBucket, ySliced, yRolled);
                            if (setup.name.compare(experiments.back().name) != 0
                                && setup.sLen.back() != sLen)
                                std::cout << "###########################################" << std::endl
                                << "wmNr=" << std::to_string(wmNr) << std::endl
                                << "ePerWm=" << std::to_string(ePerWm) << std::endl
                                << "wLen=" << std::to_string(wLen) << std::endl
                                << "sLen=" << std::to_string(sLen) << std::endl;
                        }

            PyRun_SimpleString("plt.figure()");

            std::vector<long> x;
            if (1 < setup.wmNr.size())
                x = setup.wmNr;
            else if (1 < setup.ePerWm.size())
                x = setup.ePerWm;
            else if (1 < setup.wLen.size())
                x = setup.wLen;
            else if (1 < setup.sLen.size())
                x = setup.sLen;

            std::string xString ="x=[";
            for (double item : x)
                xString += std::to_string(item) +
                (item == x.back() ? "" : ", ");
            xString += "]";
            PyRun_SimpleString(xString.c_str());

            std::string yBucketString ="bucket=[";
            for (double item : yBucket)
                yBucketString += std::to_string(item) +
                (item == yBucket.back() ? "" : ", ");
            yBucketString += "]";
            PyRun_SimpleString(yBucketString.c_str());

            std::string ySlicedString ="sliced=[";
            for (double item : ySliced)
                ySlicedString += std::to_string(item) +
                (item == ySliced.back() ? "" : ", ");
            ySlicedString += "]";
            PyRun_SimpleString(ySlicedString.c_str());

            std::string yRolledString ="rolled=[";
            for (double item : yRolled)
                yRolledString += std::to_string(item) +
                (item == yRolled.back() ? "" : ", ");
            yRolledString += "]";
            PyRun_SimpleString(yRolledString.c_str());

            PyRun_SimpleString(("plt.title('" + setup.name + "')").c_str());
            PyRun_SimpleString(("plt.xlabel('" + setup.name.substr(8) + "')").c_str());
            PyRun_SimpleString("plt.ylabel('Runtime in Seconds')");
            PyRun_SimpleString("plt.xticks(x)");

            PyRun_SimpleString("plt.plot(x, bucket, marker='x')");
            PyRun_SimpleString("plt.plot(x, sliced, marker='o')");
            PyRun_SimpleString("plt.plot(x, rolled, marker='^')");

            PyRun_SimpleString("plt.legend(['bucket', 'slicing', 'rolling'])");
            PyRun_SimpleString("plt.yscale('log')");
            PyRun_SimpleString("plt.show()");
            //PyRun_SimpleString(("plt.savefig('" + folder + "/" + setup.name + ".pdf')").c_str());
        }

        // end plot stuff
        Py_FinalizeEx();
        PyMem_RawFree(plot);
    }

    exit(EXIT_SUCCESS);
}

Overwriting /content/slicing/Experiments/main.cpp


In [9]:
!gradle build
!DYLD_LIBRARY_PATH=./Experiments/build/install/main/debug/lib LD_LIBRARY_PATH=./Experiments/build/install/main/debug/lib Experiments/build/install/main/debug/lib/Experiments



<=====--------> 42% CONFIGURING s]> :Framework<=============> 100% CONFIGURING s]> IDLE<=------------> 8% EXECUTING s]> :BucketSlidingWindow:linkDebugShared<===----------> 29% EXECUTING s]> IDLE
BUILD SUCCESSFUL in 1s
15 actionable tasks: 15 up-to-date


<-------------> 0% WAITING> IDLE
Varying Count of Watermarks
***************************
Experimental Environment Initialization
Initializing Bucket Sliding Window Experiment
Started Bucket Sliding Window Experiment
Cleaning memory
Initializing Sliced Sliding Window Experiment
Started Sliced Sliding Window Experiment
Cleaning memory
Initializing Rolling Sliding Window Experiment
Started Rolling Sliding Window Experiment
Cleaning memory
Summary
  Number of events processed: 2508659
  Bucket Sliding Window time in sec:  75.901
  Sliced Sliding Window time in sec:  5.915
  Rolling Sliding Window time in sec: 1.883
  Rolling Sliding Window is 3.141 faster than Sliced Sliding Window
Experimental Environment Initialization
Initializing Buc